# <center> Innoscape - Telecom Paristech - Projet fil rouge </center>
# <center> Mapping Groupes de produits </center>

In [2]:
# Librairies 
import pandas as pd
import numpy as np
from scipy import stats as scs
from scipy import sparse
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.stem.snowball import FrenchStemmer
import re
import random

from Preprocessing_treetagger_v3 import *
from gensim import models, corpora

from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split, cross_val_score

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 60)

#chemin = "../Data/"
chemin = "C:/Users/JN/Desktop/Arbeit/fil_rouge/github/Data/"
chemin_val = "/Users/valentinphetchanpheng/Documents/MS BGD Telecom ParisTech/Projet Fil Rouge/"

ModuleNotFoundError: No module named 'Preprocessing_treetagger_v3'

In [ ]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

## Importation des données et reformatage

In [ ]:
# Importation des données vendeurs
df_seller_origin = pd.read_csv(chemin + "t3_all_fr_totalsite_prod.csv", sep = ",")

In [ ]:
# Importation des données vendeurs
df_seller_origin = pd.read_csv(chemin + "t3_all_fr_totalsite_prod.csv", sep = ",")

#Création d'un df allégé pour le travail - on ne retient que les colonnes identifiées comme utiles pour l'analyse à ce stade
columns_to_keep = ['t3_all_fr_totalsite_prod.productfamily_seller',
                   't3_all_fr_totalsite_prod.prodcategory4_seller',
                   't3_all_fr_totalsite_prod.prodcategory3_seller',
                   't3_all_fr_totalsite_prod.prodcategory2_seller',
                   't3_all_fr_totalsite_prod.prodcategory1_seller',
                   't3_all_fr_totalsite_prod.productseller_name',
                   't3_all_fr_totalsite_prod.id_source',
                   't3_all_fr_totalsite_prod.brand_seller']

df_seller = df_seller_origin[columns_to_keep]

#on raccourcit un peu les noms des colonnes en ôtant les préfixes redondants
shortened_names = map(lambda string: string.replace('t3_all_fr_totalsite_prod.',''), columns_to_keep)
col_name_dict = dict(zip(columns_to_keep, shortened_names))
df_seller.rename(columns=col_name_dict, inplace=True)

In [ ]:
# Importation des données références catégories Innoscape
df_ref_origin = pd.read_csv(chemin + "2018.11.21_innoscape_internal_architecture.csv")
df_ref = df_ref_origin.drop("Date", axis=1)

#On garde que la colonne correspondante à la famille de produit de référence finale
columns_to_keep = ['innoscape_ref_productfamily_seller']
df_ref = df_ref[columns_to_keep]
df_ref = df_ref.drop_duplicates()

In [ ]:
df_seller = df_seller.dropna(how='all', subset=['productfamily_seller',
                                                'prodcategory4_seller',
                                                'prodcategory3_seller',
                                                'prodcategory2_seller',
                                                'prodcategory1_seller',
                                                'productseller_name'])
df_seller = df_seller.fillna("")

# Retrait des caractères spéciaux et de la ponctuation
dictionary = {'à':'a','â':'a','ä':'a',
              'é':'e','è':'e','ê':'e','ë':'e',
              'ï':'i','î':'i',
              'ô':'o','ö':'o',
              'ù':'u','û':'u',
              'ç':'c',
              ',':' ', "'":" ", ";":" ", ":":" ", "\?":" ", "!":" ", "\.":'', "  ":" ", 
              "\(":"","\)":"","\[":"","\]":"","-":" ","\+":"","/":"","\*":"","–":" ",
              "&":"et"
              }

dico1 = {",":" ", "'":" ", ";":" ", ":":" ", "\?":" ", "!":" ", "\.":'', "  ":" ", "  ":" ",
         "\(":"","\)":"","\[":"","\]":"","-":" ","\+":"","/":"","\*":"","–":" ",
         "&":"et"}

dico2 = {'à':'a','â':'a','ä':'a',
              'é':'e','è':'e','ê':'e','ë':'e',
              'ï':'i','î':'i',
              'ô':'o','ö':'o',
              'ù':'u','û':'u',
              'ç':'c'}


In [ ]:
# Importation de la liste de stop words
f = open("stop_words.txt", "r")
stop_words = set(f.read().split('\n'))

g = open("marques.txt", "r")
marques = set(g.read().split('\n'))

stop_words = stop_words | marques

In [ ]:
#preprocessing des données
# df_ref=preprocessing(df_ref,df_ref.columns, dico1, dico2, stop_words, "C:/TreeTagger")
list_new_col = ['productfamily_seller', 'prodcategory4_seller', 'prodcategory3_seller', 'prodcategory2_seller', 'prodcategory1_seller', 'productseller_name']
df_seller = preprocessing(df_seller, list_new_col, dico1, dico2, stop_words, "C:/TreeTagger")

In [ ]:
df_seller.head(5)

In [ ]:
df_seller2 = df_seller.loc[:,["productfamily_seller", "prodcategory4_seller", "prodcategory3_seller", "prodcategory2_seller", "prodcategory1_seller", "productseller_name",
                      "productfamily_seller_clean","prodcategory4_seller_clean","prodcategory3_seller_clean","prodcategory2_seller_clean","prodcategory1_seller_clean","productseller_name_clean"]]

In [ ]:
df_seller2.to_csv('df_total_big.csv', index=False)

## Import all data set (preprocessed)

In [4]:
# All data set to create embedding
df_total = pd.read_csv('df_total.csv')
df_total = df_total.fillna("")
df_total["concat"] = df_total['productfamily_seller_clean'] + ' ' + \
                     df_total['prodcategory4_seller_clean'] + ' ' + \
                     df_total['prodcategory3_seller_clean'] + ' ' + \
                     df_total['prodcategory2_seller_clean'] + ' ' + \
                     df_total['prodcategory1_seller_clean'] + ' ' + \
                     df_total['productseller_name_clean']

In [5]:
# Train set 
df_train = pd.read_csv('df_train.csv')
df_train = df_train.fillna("")
df_train["concat"] = df_train['productfamily_seller_clean'] + ' ' + \
                     df_train['prodcategory4_seller_clean'] + ' ' + \
                     df_train['prodcategory3_seller_clean'] + ' ' + \
                     df_train['prodcategory2_seller_clean'] + ' ' + \
                     df_train['prodcategory1_seller_clean'] + ' ' + \
                     df_train['productseller_name_clean']

In [6]:
# test set 
df_test = pd.read_csv('df_test.csv')
df_test = df_test.fillna("")
df_test["concat"] = df_test['productfamily_seller_clean'] + ' ' + \
                     df_test['prodcategory4_seller_clean'] + ' ' + \
                     df_test['prodcategory3_seller_clean'] + ' ' + \
                     df_test['prodcategory2_seller_clean'] + ' ' + \
                     df_test['prodcategory1_seller_clean'] + ' ' + \
                     df_test['productseller_name_clean']

## TF-IDF

In [7]:
# Transformation en tfidf avec la librairie Sklearn
vectorizer = TfidfVectorizer(lowercase=False)
tf_idf_total = sparse.csr_matrix(vectorizer.fit_transform(df_total["concat"]))
tfidf_feature_names = vectorizer.get_feature_names()

In [8]:
tf_idf_train = sparse.csr_matrix(vectorizer.transform(df_train["concat"]))
tf_idf_test = sparse.csr_matrix(vectorizer.transform(df_test["concat"]))

## Embeddings

### Fit Embedding

In [55]:
nmf = NMF(n_components=300, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tf_idf_total)

In [56]:
from sklearn.externals import joblib

joblib.dump(nmf, "nmf_300topics_base.save") 
nmf = joblib.load("nmf_300topics_base.save") 

In [57]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i] + " " + str(round(topic[i],2))
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [58]:
n_top_words = 4
print_top_words(nmf, tfidf_feature_names, n_top_words)

Topic #0: tuile 13.31 ardoise 0.4 couverture 0.22 huguenot 0.15
Topic #1: descente 4.56 gouttiere 3.38 evacuation 1.8 traitement 1.62
Topic #2: terre 8.47 cuit 8.43 tuile 5.29 couverture 3.69
Topic #3: peinture 5.12 adhesif 4.52 mastic 3.68 quincaillerie 2.12
Topic #4: electroportatif 6.59 quincaillerie 3.65 outillage 2.49 charbon 0.37
Topic #5: charpente 5.08 bardage 3.99 toiture 3.85 menuiserie 2.08
Topic #6: tondeur 4.67 tondeuse 4.67 autoporte 2.81 tailler 1.25
Topic #7: gouttiere 6.81 sol 2.17 revetement 2.06 bardage 2.05
Topic #8: gros 4.14 œuvre 4.09 voirie 2.82 bpe 2.74
Topic #9: ite 4.17 finition 1.78 enduire 1.55 platre 1.27
Topic #10: etancheite 6.3 toiture 2.16 autre 0.79 humidite 0.77
Topic #11: sans 4.72 fil 4.34 batterie 0.97 outillage 0.62
Topic #12: chauffe 4.91 eau 4.23 plomberie 1.45 chaud 1.17
Topic #13: gauche 5.73 rive 4.34 tuile 0.77 equerrer 0.45
Topic #14: chargeur 4.07 batterie 3.83 outillage 1.89 lithium 1.34
Topic #15: couverture 7.0 bardage 2.25 gros 2.07 œ

### Transform Train set/Test set

In [59]:
# Transform
df_train["nmf"] = nmf.transform(tf_idf_train).tolist()
df_test["nmf"] = nmf.transform(tf_idf_test).tolist()

In [60]:
df_train.loc[:,["nmf","label"]].to_csv("df_train_nmf300_allcat.csv", index=False)
df_test["nmf"].to_csv("df_test_nmf300_allcat.csv", index=False)

## Prediction

In [62]:
df_train = pd.read_csv("df_train_nmf300_allcat.csv")
df_test = pd.read_csv("df_test_nmf300_allcat.csv", header=None)
df_train['nmf'] = df_train['nmf'].apply(lambda x: np.array([float(element) for element in x[1:-1].split(', ')]))
df_test[0] = df_test[0].apply(lambda x: [float(element) for element in x[1:-1].split(', ')])

In [63]:
X = np.array(df_train['nmf'].apply(lambda x: np.array(x)).tolist())
X_val = np.array(df_test[0].apply(lambda x: np.array(x)).tolist())
y = df_train.label.values

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

### MODELS

In [ ]:
XGB = xgb.XGBClassifier(max_depth=10, n_estimators=100,n_jobs=-1)
XGB.fit(X_train, y_train)
accuracy_score(XGB.predict(X_test), y_test)

In [ ]:
LGB = lgb.LGBMClassifier(num_leaves=76, max_depth=7, n_estimators=1000, n_jobs=-1, silent = True, device_type='gpu')
LGB.fit(X_train, y_train, verbose=100, eval_metric='logloss', eval_set=[(X_test,  y_test)])
accuracy_score(LGB.predict(X_test), y_test)

In [ ]:
clf_ex = ExtraTreesClassifier(max_depth=100, random_state=2, n_estimators=2000)
clf_ex.fit(X_train, y_train)
accuracy_score(clf_ex.predict(X_test), y_test)

### RF - EX

In [65]:
clf_rf = RandomForestClassifier(max_depth=100, random_state=42, n_estimators=2000)
clf_rf.fit(X_train, y_train)
accuracy_score(clf_rf.predict(X_test), y_test)

0.8253164556962025

In [66]:
clf_ex = ExtraTreesClassifier(max_depth=100, random_state=42, n_estimators=2000)
clf_ex.fit(X_train, y_train)
accuracy_score(clf_ex.predict(X_test), y_test)

0.8455696202531645

In [67]:
clf_rf = RandomForestClassifier(max_depth=100, random_state=42, n_estimators=2000)
scores = cross_val_score(clf_rf, X, y, cv=10) 
print(scores, np.mean(scores))

C:\Users\Pascal\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


[0.70833333 0.75       0.76818182 0.75233645 0.75       0.75257732
 0.7740113  0.8255814  0.85276074 0.72258065] 0.7656362995842386


In [68]:
clf_ex = ExtraTreesClassifier(max_depth=100, random_state=42, n_estimators=2000)
scores = cross_val_score(clf_ex, X, y, cv=10) 
print(scores, np.mean(scores))

C:\Users\Pascal\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


[0.73333333 0.79310345 0.79090909 0.76635514 0.76960784 0.77835052
 0.81355932 0.81395349 0.87730061 0.71612903] 0.7852601827467363


RF 200 topics :    
base :  0.7754672213718672 (10CV) / 0.8405063291139241           
big : 0.7625318850009865 (10CV) / 0.8227848101265823          

EX 200 topics :    
base : 0.8028580068538258 (10CV) / 0.8734177215189873               
big : 0.7777444057525182 (10CV) / 0.8455696202531645         

RF 300 topics :    
base :  0.7656362995842386 (10CV) / 0.8253164556962025       

EX 300 topics :    
base : 0.7852601827467363 (10CV) / 0.8455696202531645                          

RF 500 topics :    
base : 0.7731587262576494 (10CV) / 0.8278481012658228          
big : 0.776486561 (10CV) / 0.830379746835443    

EX 500 topics :    
base : 0.7895499011627617 (10CV) / 0.850632911392405            
big : 0.786398517 (10CV) / 0.8379746835443038     

In [26]:
# Calcul des Features Importances pour la longitude
dict_feature_importance = {'feature': range(500), 'importance': clf_rf.feature_importances_}
feature_importances = pd.DataFrame(data=dict_feature_importance).sort_values('importance', ascending=False)

# Calcul des Features Importances pour la latitude
plt.figure(figsize=(20,40))
sns.barplot(x="importance",
            y="feature",
            orient='h',
            data=feature_importances.sort_values(by="importance",
                                                 ascending=False))
plt.title('Feature Importance')
plt.tight_layout()

In [27]:
topics_to_drop = feature_importances[feature_importances.importance == 0].loc[:, 'feature'].values

In [28]:
len(topics_to_drop)

352

In [32]:
topics_to_keep = list(set(range(500)) - set(topics_to_drop)) + [72]

In [33]:
X_train2 = X_train[:, topics_to_keep]
X_test2 = X_test[:, topics_to_keep]

In [34]:
clf_rf = RandomForestClassifier(max_depth=100, random_state=2, n_estimators=2000)
clf_rf.fit(X_train2, y_train)
accuracy_score(clf_rf.predict(X_test2), y_test)

0.8151898734177215

RF (prodcat4/productfamily/productname):       
500 topics : 0.7873417721518987 / 0.789873417721519 (seuil importance 0.0005)          
200 topics : 0.7772151898734178  / 0.779746835443038 (seuil importance 0.001)  
100 topics : 0.7189873417721518      
50 topics : 0.6886075949367089

RF (prodcat4-3-2-1/productfamily/productname):               
200 topics : 0.8405063291139241  / 0.8379746835443038 (seuil importance 0.001)           
500 topics : 0.8278481012658228      

RF (alldata + prodcat4-3-2-1/productfamily/productname):       
200 topics : 0.8227848101265823         

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def print_confusion_matrix(confusion_matrix, class_names, figsize = (25,15), fontsize=14):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names, 
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return fig

In [ ]:
prediction = clf_rf.predict(X_test2).tolist()

In [ ]:
labels = []
labels = prediction + y_test.values.tolist()
labels = list(set(labels))

In [ ]:
confusion_matrix(y_test, clf_rf.predict(X_test2), )

In [ ]:
print_confusion_matrix(confusion_matrix(y_test, clf_rf.predict(X_test2)), labels)

## Validate on the rest of the dataset

In [ ]:
prediction = clf_rf.predict(X_val).tolist()

In [ ]:
df_test['label'] = prediction

In [ ]:
df_test.iloc[:,1:].to_csv('validation_set.csv', index=False)